In [3]:
import numpy as np
from tensorflow import keras

from lib.sin_cos_layer import SinCos
from lib.utils import save_keras_model

import tensorflow.keras.backend as K

In [4]:
i_m = keras.layers.Input(shape=(1,), dtype="int32", name="i_m")
i_y = keras.layers.Input(shape=(1,), dtype="int32", name="i_y")
f_m_sincos = SinCos(val_range=[1, 12], name="f_m_sincos")(i_m)
f_date = keras.layers.Lambda(
    lambda x: K.cast(x[1] * 12 + x[0], 'float32'),
)([i_y, i_m])

m = keras.models.Model(inputs=[i_y, i_m], outputs=[f_m_sincos, f_date])

In [5]:
m.predict(x=[{"i_y": np.array([2021]), "i_m": np.array([5])}])

[array([[-0.75574957, -0.65486073]]), array([[24257.]], dtype=float32)]

In [6]:
save_keras_model(m, 'model.png')